In [1]:
import gc
import os
import ast
import cv2
import time
import timm
# import timm4smp
import pickle
import random
import pydicom
import argparse
import warnings
import threading
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from glob import glob
import albumentations
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from pylab import rcParams

%matplotlib inline
device = torch.device('cuda')
torch.backends.cudnn.benchmark = True

In [2]:
class CFG:
    data_dir = 'data'
    image_size_seg = (128, 128, 128)
    msk_size = image_size_seg[0]
    backbone_seg = 'resnet18'
    drop_rate = 0.
    drop_path_rate = 0.
    n_blocks = 4
    out_dim_seg = 5
    
    # cls model
    in_chans = 6
    backbone_cls = 'tf_efficientnetv2_s_in21ft1k'
    image_size_cls = 224
    n_slice_per_c = 15
    n_ch = 5
    out_dim = 1
    dropout = 0.1
    batch_size_seg = 1
    num_workers = 2
    device = 'cuda'
    seed = 42

In [3]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    print('seeding done!!!')
seeding(CFG.seed)

seeding done!!!


In [4]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = cv2.resize(data, (CFG.image_size_seg[0], CFG.image_size_seg[1]), interpolation = cv2.INTER_LINEAR)
    return data

def load_dicom_line_par(path):

    t_paths = sorted(glob(os.path.join(path, "*")), key=lambda x: int(x.split('/')[-1].split(".")[0]))
    
    n_scans = len(t_paths)
    
    indices = np.quantile(list(range(n_scans)), np.linspace(0., 1., CFG.image_size_seg[2])).round().astype(int)
    t_paths = [t_paths[i] for i in indices]

    images = []
    for filename in t_paths:
        images.append(load_dicom(filename))
    images = np.stack(images, -1)
    
    images = images - np.min(images)
    images = images / (np.max(images) + 1e-4)
    images = (images * 255).astype(np.uint8)

    return images

class SegTestDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index()
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        image = load_dicom_line_par(row.dicom_folder)
        if image.ndim < 4:
            image = np.expand_dims(image, 0)
        image = image.astype(np.float32).repeat(3, 0)
        image = image / 255.
        return torch.tensor(image).float()


In [5]:
import os
import numpy as np
import pandas as pd


def prepare_folds(data_path="data/", k=20):
    """
    Prepare data folds for cross-validation.
    MultilabelStratifiedKFold is used.

    Args:
        data_path (str, optional): Path to the data directory. Defaults to "../input/".
        k (int, optional): Number of cross-validation folds. Defaults to 4.

    Returns:
        pandas DataFrame: DataFrame containing the patient IDs and their respective fold assignments.
    """
    cols = [
        'bowel_injury', 'extravasation_injury', 'kidney_low',
        'kidney_high', 'liver_low', 'liver_high', 'spleen_low', 'spleen_high'
    ]

    df = pd.read_csv(data_path + "train.csv")

    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    mskf = MultilabelStratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    splits = mskf.split(df, y=df[cols])

    df['fold'] = -1
    for i, (_, val_idx) in enumerate(splits):
        df.loc[val_idx, "fold"] = i

    df_folds = df[["patient_id", "fold"]]
    # df_folds.to_csv(data_path + f"stage1/folds_{k}.csv", index=False)
    return df_folds


df = prepare_folds()
valid_ = df[df['fold'] == 0].reset_index(drop=True)
print(valid_.head())

   patient_id  fold
0       10172     0
1       11301     0
2       11378     0
3       12299     0
4       13106     0


In [34]:
# valid_.to_csv('data/stage1/fold0_offical.csv', index=False)

In [6]:
def is_folder_empty(folder_path):
    return os.path.exists(folder_path) == 0

df = pd.read_csv(os.path.join(CFG.data_dir, 'train_series_meta.csv'))
df['dicom_folder'] = CFG.data_dir + '/train_images/' + df.patient_id.astype(str) + '/' + df.series_id.astype(str)

df['IsEmpty'] = df['dicom_folder'].apply(is_folder_empty)
df = df[~df['IsEmpty']]
df = df.drop(columns=['IsEmpty'])
df = df.reset_index()
df = pd.merge(df, valid_, on='patient_id', how='inner')
print(len(df))
print(df.head())
# df = df.merge(valid_, on='series_id', how='left')
# print(df.head())

237
   index  patient_id  series_id  aortic_hu  incomplete_organ  \
0     19       10172      50253      158.0                 0   
1     93       11301       2703      165.0                 0   
2     94       11301      57412      230.0                 0   
3    100       11378      16846      465.0                 0   
4    101       11378      53429      128.0                 0   

                    dicom_folder  fold  
0  data/train_images/10172/50253     0  
1   data/train_images/11301/2703     0  
2  data/train_images/11301/57412     0  
3  data/train_images/11378/16846     0  
4  data/train_images/11378/53429     0  


In [7]:
dataset_seg = SegTestDataset(df)
loader_seg = torch.utils.data.DataLoader(dataset_seg, batch_size=CFG.batch_size_seg, shuffle=False, num_workers=CFG.num_workers)

In [8]:
import torch.nn as nn
import timm
import segmentation_models_pytorch as smp
from timm.layers import Conv2dSame
# from conv3d_same import Conv3dSame


class TimmSegModel(nn.Module):
    def __init__(self, backbone, segtype='unet', pretrained=False):
        super(TimmSegModel, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            drop_rate=CFG.drop_rate,
            drop_path_rate=CFG.drop_path_rate,
            pretrained=pretrained
        )
        g = self.encoder(torch.rand(1, 3, 64, 64))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        n_blocks = CFG.n_blocks
        if segtype == 'unet':
            self.decoder = smp.decoders.unet.decoder.UnetDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], CFG.out_dim_seg, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:CFG.n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features


def convert_3d(module):
    module_output = module
    if isinstance(module, torch.nn.BatchNorm2d):
        module_output = torch.nn.BatchNorm3d(
            module.num_features,
            module.eps,
            module.momentum,
            module.affine,
            module.track_running_stats,
        )
        if module.affine:
            with torch.no_grad():
                module_output.weight = module.weight
                module_output.bias = module.bias
        module_output.running_mean = module.running_mean
        module_output.running_var = module.running_var
        module_output.num_batches_tracked = module.num_batches_tracked
        if hasattr(module, "qconfig"):
            module_output.qconfig = module.qconfig
            
    # elif isinstance(module, Conv2dSame):
    #     module_output = Conv3dSame(
    #         in_channels=module.in_channels,
    #         out_channels=module.out_channels,
    #         kernel_size=module.kernel_size[0],
    #         stride=module.stride[0],
    #         padding=module.padding[0],
    #         dilation=module.dilation[0],
    #         groups=module.groups,
    #         bias=module.bias is not None,
    #     )
    #     module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.Conv2d):
        module_output = torch.nn.Conv3d(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
            padding_mode=module.padding_mode
        )
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.MaxPool2d):
        module_output = torch.nn.MaxPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            dilation=module.dilation,
            ceil_mode=module.ceil_mode,
        )
    elif isinstance(module, torch.nn.AvgPool2d):
        module_output = torch.nn.AvgPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            ceil_mode=module.ceil_mode,
        )

    for name, child in module.named_children():
        module_output.add_module(
            name, convert_3d(child)
        )
    del module

    return module_output


# load model
model_seg_path = 'weights'
seg_model = TimmSegModel(CFG.backbone_seg, pretrained=False)
seg_model = convert_3d(seg_model)
seg_model = seg_model.to(CFG.device)
load_model_file = os.path.join(model_seg_path, 'resnet18_fold0_best.pth')
sd = torch.load(load_model_file)
if 'model_state_dict' in sd.keys():
    sd = sd['model_state_dict']
sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
seg_model.load_state_dict(sd, strict=True)
seg_model.eval()
print('load segmodel successfull!')

load segmodel successfull!


In [9]:
from itertools import repeat


class SpatialDropout(nn.Module):
    def __init__(self, drop=0.5):
        super(SpatialDropout, self).__init__()
        self.drop = drop
        
    def forward(self, inputs, noise_shape=None):
        """
        @param: inputs, tensor
        @param: noise_shape, tuple
        """
        outputs = inputs.clone()
        if noise_shape is None:
            noise_shape = (inputs.shape[0], *repeat(1, inputs.dim()-2), inputs.shape[-1]) 
        
        self.noise_shape = noise_shape
        if not self.training or self.drop == 0:
            return inputs
        else:
            noises = self._make_noises(inputs)
            if self.drop == 1:
                noises.fill_(0.0)
            else:
                noises.bernoulli_(1 - self.drop).div_(1 - self.drop)
            noises = noises.expand_as(inputs)    
            outputs.mul_(noises)
            return outputs
            
    def _make_noises(self, inputs):
        return inputs.new().resize_(self.noise_shape)


class MLPAttentionNetwork(nn.Module):
 
    def __init__(self, hidden_dim, attention_dim=None):
        super(MLPAttentionNetwork, self).__init__()
 
        self.hidden_dim = hidden_dim
        self.attention_dim = attention_dim
        if self.attention_dim is None:
            self.attention_dim = self.hidden_dim
        # W * x + b
        self.proj_w = nn.Linear(self.hidden_dim, self.attention_dim, bias=True)
        # v.T
        self.proj_v = nn.Linear(self.attention_dim, 1, bias=False)
 
    def forward(self, x):
        """
        :param x: seq_len, batch_size, hidden_dim
        :return: batch_size * seq_len, batch_size * hidden_dim
        """
        # print(f"x shape:{x.shape}")
        batch_size, seq_len, _ = x.size()
        # flat_inputs = x.reshape(-1, self.hidden_dim) # (batch_size*seq_len, hidden_dim)
        # print(f"flat_inputs shape:{flat_inputs.shape}")
        
        H = torch.tanh(self.proj_w(x)) # (batch_size, seq_len, hidden_dim)
        # print(f"H shape:{H.shape}")
        
        att_scores = torch.softmax(self.proj_v(H),axis=1) # (batch_size, seq_len)
        # print(f"att_scores shape:{att_scores.shape}")
        
        attn_x = (x * att_scores).sum(1) # (batch_size, hidden_dim)
        # print(f"attn_x shape:{attn_x.shape}")
        return attn_x

class TimmModel2(nn.Module):
    def __init__(self, backbone, pretrained=False):
        super().__init__()
        self.encoder = timm.create_model(
            backbone,
            in_chans=CFG.in_chans,
            num_classes=CFG.out_dim,
            features_only=False,
            drop_rate=CFG.drop_rate,
            drop_path_rate=CFG.drop_path_rate,
            pretrained=pretrained
        )
        
        hdim = self.encoder.conv_head.out_channels
        self.encoder.classifier = nn.Identity()
        
        self.spatialdropout = SpatialDropout(CFG.dropout)
        self.gru = nn.GRU(hdim, 256, 2, batch_first=True, bidirectional=True)
        self.mlp_attention_layer = MLPAttentionNetwork(512)
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(0),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 1),
        )

    def forward(self, x):  # (bs, nslice, ch, sz, sz)
        bs = x.shape[0]
        x = x.view(bs * CFG.n_slice_per_c, CFG.in_chans, CFG.image_size_cls, CFG.image_size_cls)
        feat = self.encoder(x)
        feat = self.spatialdropout(feat)
        feat = feat.view(bs, CFG.n_slice_per_c, -1)
        feat, _ = self.gru(feat)
        feat = self.mlp_attention_layer(feat) # [bs, 512]
        # feat = feat.contiguous().view(bs * CFG.n_slice_per_c, -1)
        feat = self.head(feat)
        # feat = feat.view(bs, CFG.n_slice_per_c).contiguous()
        # feat = self.mean_layer(feat)
        feat = feat.view(bs, -1)
        return feat


# load cls model
model_cls_path = 'weights'
cls_models = []

for idx_fold in range(1):
    cls_model = TimmModel2(CFG.backbone_cls, pretrained=False)
    load_cls_model_file = os.path.join(model_cls_path, f'filter_stage2_tf_efficientnetv2_s_in21ft1k_fold{idx_fold}_last.pth')
    sd = torch.load(load_cls_model_file, map_location='cpu')
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    cls_model.load_state_dict(sd, strict=True)
    cls_model = cls_model.to(device)
    cls_model.eval()
    cls_models.append(cls_model)
print(f'load cls_model succesfull for {len(cls_models)} models!')


# class Timm1BoneModel(nn.Module):
#     def __init__(self, backbone, image_size, pretrained=False):
#         super(Timm1BoneModel, self).__init__()
#         self.image_size = image_size

#         self.encoder = timm.create_model(
#             backbone,
#             in_chans=CFG.in_chans,
#             num_classes=1,
#             features_only=False,
#             drop_rate=0,
#             drop_path_rate=0,
#             pretrained=pretrained
#         )

#         if 'efficient' in backbone:
#             hdim = self.encoder.conv_head.out_channels
#             self.encoder.classifier = nn.Identity()
#         elif 'convnext' in backbone or 'nfnet' in backbone:
#             hdim = self.encoder.head.fc.in_features
#             self.encoder.head.fc = nn.Identity()

#         self.lstm = nn.LSTM(hdim, 256, num_layers=2, dropout=0, bidirectional=True, batch_first=True)
#         self.head = nn.Sequential(
#             nn.Linear(512, 256),
#             nn.BatchNorm1d(256),
#             nn.Dropout(0),
#             nn.LeakyReLU(0.1),
#             nn.Linear(256, 1),
#         )


#     def forward(self, x):  # (bs, nslice, ch, sz, sz)
#         bs = x.shape[0]
#         x = x.view(bs * CFG.n_slice_per_c, CFG.in_chans, self.image_size_cls, self.image_size_cls)
#         feat = self.encoder(x)
#         feat = feat.view(bs, CFG.n_slice_per_c, -1)
#         feat, _ = self.lstm(feat)
#         feat = feat.contiguous().view(bs * CFG.n_slice_per_c, -1)
#         feat = self.head(feat)
#         feat = feat.view(bs, CFG.n_slice_per_c).contiguous()

#         return feat


# load cls model
# model_cls_path = 'weights'
# cls_models = []

# for idx_fold in range(1):
#     cls_model = Timm2BoneModel(CFG.backbone_cls, pretrained=False)
#     load_cls_model_file = os.path.join(model_cls_path, f'filter_stage2_tf_efficientnetv2_s_in21ft1k_fold{idx_fold}_best078.pth')
#     sd = torch.load(load_cls_model_file, map_location='cpu')
#     if 'model_state_dict' in sd.keys():
#         sd = sd['model_state_dict']
#     sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
#     cls_model.load_state_dict(sd, strict=True)
#     cls_model = cls_model.to(device)
#     cls_model.eval()
#     cls_models.append(cls_model)
# print(f'load cls_model succesfull for {len(cls_models)} models!')

/data1/ocr_ie/anaconda3/envs/rsna/lib/python3.8/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21ft1k to current tf_efficientnetv2_s.in21k_ft_in1k.
  model = create_fn(


load cls_model succesfull for 1 models!


In [11]:
output = []

a = time.time()
with torch.no_grad():
    for batch_id, images in enumerate(loader_seg):
        # if batch_id < 96:
        #     continue
        print('start with index {}'.format(batch_id))
        images = images.cuda()
        # seg
        pmask = seg_model(images).sigmoid()
        pmask = pmask.squeeze(0)
   
        mask = pmask.detach().cpu().numpy()
        paths = df.loc[batch_id, 'dicom_folder']
        
        study_id = paths.split('/')[-1]
        patient_id = paths.split('/')[-2]
        t_paths = sorted(glob(os.path.join(paths, "*")), key=lambda x: int(x.split('/')[-1].split(".")[0]))
        n_scans = len(t_paths)
        print(f'path {paths} with total {n_scans} scan')
        cls_inp = []
        cropped_images = [None] * 5
        
        for cid in range(5):
            bone = []
            try:
                msk_b = mask[cid] > 0.2
                msk_c = mask[cid] > 0.05
                x = np.where(msk_b.sum(1).sum(1) > 0)[0]
                y = np.where(msk_b.sum(0).sum(1) > 0)[0]
                z = np.where(msk_b.sum(0).sum(0) > 0)[0]
    
                if len(x) == 0 or len(y) == 0 or len(z) == 0:
                    x = np.where(msk_c.sum(1).sum(1) > 0)[0]
                    y = np.where(msk_c.sum(0).sum(1) > 0)[0]
                    z = np.where(msk_c.sum(0).sum(0) > 0)[0]
    
                x1, x2 = max(0, x[0] - 1), min(mask.shape[1], x[-1] + 1)
                y1, y2 = max(0, y[0] - 1), min(mask.shape[2], y[-1] + 1)
                z1, z2 = max(0, z[0] - 1), min(mask.shape[3], z[-1] + 1)
                zz1, zz2 = int(z1 / CFG.msk_size * n_scans), int(z2 / CFG.msk_size * n_scans)
                inds = np.linspace(zz1 ,zz2-1, CFG.n_slice_per_c).astype(int)
                inds_ = np.linspace(z1 ,z2-1, CFG.n_slice_per_c).astype(int)
                for sid, (ind, ind_) in enumerate(zip(inds, inds_)):
                    
                    msk_this = mask[cid, :, :, ind_]
                    images = []
                    for i in range(-CFG.n_ch // 2 + 1, CFG.n_ch // 2 + 1):
                        try:
                            dicom = pydicom.read_file(t_paths[ind+i])
                            images.append(dicom.pixel_array)
                        except:
                            images.append(np.zeros((512, 512)))
                    data = np.stack(images, -1)
                    data = data - np.min(data)
                    data = data / (np.max(data) + 1e-4)
                    data = (data * 255).astype(np.uint8)
                    
                    msk_this = msk_this[x1:x2, y1:y2]
                    xx1 = int(x1 / CFG.msk_size * data.shape[0])
                    xx2 = int(x2 / CFG.msk_size * data.shape[0])
                    yy1 = int(y1 / CFG.msk_size * data.shape[1])
                    yy2 = int(y2 / CFG.msk_size * data.shape[1])
    
                    data = data[xx1:xx2, yy1:yy2]
                    data = np.stack([cv2.resize(data[:, :, i], (CFG.image_size_cls, CFG.image_size_cls), interpolation = cv2.INTER_LINEAR) for i in range(CFG.n_ch)], -1)
                    msk_this = (msk_this * 255).astype(np.uint8)
                    msk_this = cv2.resize(msk_this, (CFG.image_size_cls, CFG.image_size_cls), interpolation = cv2.INTER_LINEAR)
    
                    data = np.concatenate([data, msk_this[:, :, np.newaxis]], -1)
                    bone.append(torch.tensor(data))
            except:
                bone = []
                print('except process fail', cid)
                for sid in range(CFG.n_slice_per_c):
                    bone.append(torch.zeros((CFG.image_size_cls, CFG.image_size_cls, CFG.n_ch + 1)).int())
            # print('bone', len(bone))
            cropped_images[cid] = torch.stack(bone, 0)

        cropped_images = torch.cat(cropped_images, 0)
        print('cropped_images', cropped_images.size())
        cls_inp = cropped_images.permute(0, 3, 1, 2).float() / 255.
        cls_inp = cls_inp.to(CFG.device)
        # cls_inp = cls_inp.unsqueeze(0) # (1, 15*5, 6, 224, 224)
        cls_inp = cls_inp.view(5, 15, 6, CFG.image_size_cls, CFG.image_size_cls).contiguous()
        pred_cls = []
        for _, model in enumerate(cls_models):
            logits = model(cls_inp)
            print('logits', logits)
            # logits = logits.sigmoid().view(-1, 5, CFG.n_slice_per_c)
            logits = logits.sigmoid().view(-1, 5)
            print(logits)
            pred_cls.append(logits)
        pred_cls = torch.stack(pred_cls, 0).mean(0)
        output.append(pred_cls.cpu())
        # logits = cls_model(cls_inp)
        # print('logits', logits.size())
        # logits = logits.sigmoid().view(-1, 5, CFG.n_slice_per_c)
        # output.append(logits.cpu())

map_cls = {0: 'liver', 1: 'spleen', 2: 'kidney', 3: 'kidney', 4: 'bowel'}
print(time.time() - a)

start with index 0
path data/train_images/10172/50253 with total 394 scan
cropped_images torch.Size([75, 224, 224, 6])
logits tensor([[ 0.4385],
        [ 0.5184],
        [-0.5392],
        [-5.5657],
        [-0.4836]], device='cuda:0')
tensor([[0.6079, 0.6268, 0.3684, 0.0038, 0.3814]], device='cuda:0')
start with index 1
path data/train_images/11301/2703 with total 170 scan
cropped_images torch.Size([75, 224, 224, 6])
logits tensor([[-5.1624],
        [ 3.1177],
        [-3.1212],
        [-3.7280],
        [-6.4215]], device='cuda:0')
tensor([[0.0057, 0.9576, 0.0422, 0.0235, 0.0016]], device='cuda:0')
start with index 2
path data/train_images/11301/57412 with total 116 scan
cropped_images torch.Size([75, 224, 224, 6])
logits tensor([[-4.3892],
        [ 3.5958],
        [-1.9903],
        [-4.0156],
        [-6.1013]], device='cuda:0')
tensor([[0.0123, 0.9733, 0.1202, 0.0177, 0.0022]], device='cuda:0')
start with index 3
path data/train_images/11378/16846 with total 571 scan
croppe

In [18]:
import json
with open('data/train_dict.json', 'r') as fr:
    aohus = json.load(fr)


def norm_to_one(x):
    s = sum(x)
    x=[xx/s for xx in x]
    return x


def post_process(preds, df):
    all = []
    # weight mean extra
    train = pd.read_csv(f'{CFG.data_dir}/train.csv')
    features_bowel = ['bowel_healthy', 'bowel_injury']
    features = ['extravasation_healthy', 'extravasation_injury']
    
    extra = train[features].mean().tolist()
    extra = norm_to_one([extra[0], extra[1] * 6])
    # bowel = train[features_bowel].mean().tolist()
    # bowel = norm_to_one([bowel[0], bowel[1] * 6])
    print('extravasation value', extra)
    # print('bowel value', bowel)
    # extra[1] = extra[1] * 6
    # extra[2] = extra[2] * 28
    # for idx, (pred, aohu) in enumerate(zip(preds, aohus)):
    #     if str(aohu) in data_aohu.keys():
    #         aohu_weight = data_aohu[str(aohu)]
    #     else:
    #         aohu_weight = 1
    for idx, pred in enumerate(preds):
        liver_pred, spleen_pred, kidney_left_pred, kidney_right_pred, bowel_pred = pred
        liver_healthy, liver_low, liver_high = (1 - liver_pred), liver_pred / 6, liver_pred / 6
        # liver_healthy, liver_low, liver_high = 0.897998, 0.223392, 0.019701
        spleen_healthy, spleen_low, spleen_high = (1 - spleen_pred), spleen_pred / 6, spleen_pred / 6
        # spleen_healthy, spleen_low, spleen_high = 0.887512, 0.171641, 0.049253
        kidney_pred = (kidney_left_pred + kidney_right_pred) / 2
        kidney_healthy, kidney_low, kidney_high = (1 - kidney_pred), kidney_pred / 6, kidney_pred / 6
        # kidney_healthy, kidney_low, kidney_high = 0.942167, 0.099189, 0.141749
        bowel_healthy, bowel_injury = (1 - bowel_pred), bowel_pred / 2
        # bowel_healthy, bowel_injury = 0.979663, 0.135402
        all.append([df.loc[idx, 'patient_id'], bowel_healthy, bowel_injury, extra[0], extra[1], kidney_healthy,
                    kidney_low, kidney_high, liver_healthy, liver_low, liver_high,
                    spleen_healthy, spleen_low, spleen_high])
    return all

In [19]:

mapping = {0: 'liver', 1: 'spleen', 2: 'kidney', 3: 'kidney', 4: 'bowel'}
outputs = torch.cat(output)
preds = outputs

# preds, _ = torch.max(outputs, dim=2)
# print(preds)
# preds = (outputs.mean(-1)).clamp(0.0001, 0.9999)
preds = preds.detach().cpu().numpy()
preds = post_process(preds, df)

extravasation value [0.7106341933928141, 0.2893658066071859]


In [20]:
sub_df = pd.DataFrame(preds, columns=['patient_id', 'bowel_healthy', 'bowel_injury', 'extravasation_healthy', 'extravasation_injury',
                                      'kidney_healthy', 'kidney_low', 'kidney_high', 'liver_healthy', 'liver_low', 'liver_high',
                                      'spleen_healthy', 'spleen_low', 'spleen_high'])
sub_df = sub_df.sort_values(by='patient_id')
sub_df.head(5)



,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
73,263,0.996290,0.001855,0.710634,0.289366,0.896853,0.017191,0.017191,0.949229,0.008462,0.008462,0.996306,0.000616,0.000616
134,394,0.998264,0.000868,0.710634,0.289366,0.499183,0.083470,0.083470,0.988489,0.001919,0.001919,0.989036,0.001827,0.001827
154,470,0.996143,0.001928,0.710634,0.289366,0.985217,0.002464,0.002464,0.327805,0.112033,0.112033,0.644929,0.059178,0.059178
153,470,0.999183,0.000409,0.710634,0.289366,0.949890,0.008352,0.008352,0.887727,0.018712,0.018712,0.905172,0.015805,0.015805
228,766,0.996414,0.001793,0.710634,0.289366,0.913907,0.014349,0.014349,0.996829,0.000528,0.000528,0.471110,0.088148,0.088148


In [21]:
import json
# with open('data/train_dict.json', 'w') as fw:
    
sub_df = sub_df.groupby('patient_id').mean().reset_index()
# sub_df = sub_df.fillna(0.5)

In [22]:
sub_df.head(50)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,263,0.996290,0.001855,0.710634,0.289366,0.896853,0.017191,0.017191,0.949229,0.008462,0.008462,0.996306,0.000616,0.000616
1,394,0.998264,0.000868,0.710634,0.289366,0.499183,0.083470,0.083470,0.988489,0.001919,0.001919,0.989036,0.001827,0.001827
2,470,0.997663,0.001169,0.710634,0.289366,0.967554,0.005408,0.005408,0.607766,0.065372,0.065372,0.775050,0.037492,0.037492
3,766,0.997165,0.001417,0.710634,0.289366,0.929134,0.011811,0.011811,0.995938,0.000677,0.000677,0.537997,0.077000,0.077000
4,878,0.979931,0.010035,0.710634,0.289366,0.943247,0.009459,0.009459,0.960277,0.006620,0.006620,0.386555,0.102241,0.102241
5,1900,0.998812,0.000594,0.710634,0.289366,0.996052,0.000658,0.000658,0.013177,0.164471,0.164471,0.958792,0.006868,0.006868
6,2288,0.985826,0.007087,0.710634,0.289366,0.341038,0.109827,0.109827,0.742488,0.042919,0.042919,0.102959,0.149507,0.149507
7,2940,0.959369,0.020315,0.710634,0.289366,0.894200,0.017633,0.017633,0.914503,0.014249,0.014249,0.946050,0.008992,0.008992
8,2978,0.982794,0.008603,0.710634,0.289366,0.880377,0.019937,0.019937,0.773608,0.037732,0.037732,0.626991,0.062168,0.062168
9,3881,0.998970,0.000515,0.710634,0.289366,0.939118,0.010147,0.010147,0.944011,0.009331,0.009331,0.847058,0.025490,0.025490


In [23]:
# metrics
import sklearn
class ParticipantVisibleError(Exception):
    pass


def normalize_probabilities_to_one(df: pd.DataFrame, group_columns: list) -> pd.DataFrame:
    # Normalize the sum of each row's probabilities to 100%.
    # 0.75, 0.75 => 0.5, 0.5
    # 0.1, 0.1 => 0.5, 0.5
    row_totals = df[group_columns].sum(axis=1)
    if row_totals.min() == 0:
        raise ParticipantVisibleError('All rows must contain at least one non-zero prediction')
    for col in group_columns:
        df[col] /= row_totals
    return df


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    '''
    Pseudocode:
    1. For every label group (liver, bowel, etc):
        - Normalize the sum of each row's probabilities to 100%.
        - Calculate the sample weighted log loss.
    2. Derive a new any_injury label by taking the max of 1 - p(healthy) for each label group
    3. Calculate the sample weighted log loss for the new label group
    4. Return the average of all of the label group log losses as the final score.
    '''
    del solution[row_id_column_name]
    # del submission[row_id_column_name]

    # Run basic QC checks on the inputs
    if not pd.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('All submission values must be numeric')

    if not np.isfinite(submission.values).all():
        raise ParticipantVisibleError('All submission values must be finite')

    if solution.min().min() < 0:
        raise ParticipantVisibleError('All labels must be at least zero')
    if submission.min().min() < 0:
        raise ParticipantVisibleError('All predictions must be at least zero')

    # Calculate the label group log losses
    binary_targets = ['bowel', 'extravasation']
    triple_level_targets = ['kidney', 'liver', 'spleen']
    all_target_categories = binary_targets + triple_level_targets

    label_group_losses = []
 
    for category in all_target_categories:
        
        if category in binary_targets:
            col_group = [f'{category}_healthy', f'{category}_injury']
        else:
            col_group = [f'{category}_healthy', f'{category}_low', f'{category}_high']

        solution = normalize_probabilities_to_one(solution, col_group)

        for col in col_group:
            if col not in submission.columns:
                raise ParticipantVisibleError(f'Missing submission column {col}')
        submission = normalize_probabilities_to_one(submission, col_group)

        label_group_losses.append(
            sklearn.metrics.log_loss(
                y_true=solution[col_group].values,
                y_pred=submission[col_group].values,
                sample_weight=solution[f'{category}_weight'].values
            )
        )

    # Derive a new any_injury label by taking the max of 1 - p(healthy) for each label group
    healthy_cols = [x + '_healthy' for x in all_target_categories]
    any_injury_labels = (1 - solution[healthy_cols]).max(axis=1)
    any_injury_predictions = (1 - submission[healthy_cols]).max(axis=1)
    any_injury_loss = sklearn.metrics.log_loss(
        y_true=any_injury_labels.values,
        y_pred=any_injury_predictions.values,
        sample_weight=solution['any_injury_weight'].values
    )

    label_group_losses.append(any_injury_loss)
    return np.mean(label_group_losses)


# Assign the appropriate weights to each category
def create_training_solution(y_train):
    sol_train = y_train.copy()
    
    # bowel healthy|injury sample weight = 1|2
    sol_train['bowel_weight'] = np.where(sol_train['bowel_injury'] == 1, 2, 1)
    
    # extravasation healthy/injury sample weight = 1|6
    sol_train['extravasation_weight'] = np.where(sol_train['extravasation_injury'] == 1, 6, 1)
    
    # kidney healthy|low|high sample weight = 1|2|4
    sol_train['kidney_weight'] = np.where(sol_train['kidney_low'] == 1, 2, np.where(sol_train['kidney_high'] == 1, 4, 1))
    
    # liver healthy|low|high sample weight = 1|2|4
    sol_train['liver_weight'] = np.where(sol_train['liver_low'] == 1, 2, np.where(sol_train['liver_high'] == 1, 4, 1))
    
    # spleen healthy|low|high sample weight = 1|2|4
    sol_train['spleen_weight'] = np.where(sol_train['spleen_low'] == 1, 2, np.where(sol_train['spleen_high'] == 1, 4, 1))
    
    # any healthy|injury sample weight = 1|6
    sol_train['any_injury_weight'] = np.where(sol_train['any_injury'] == 1, 6, 1)
    return sol_train


df1 = pd.read_csv('data/train.csv')
df1 = pd.merge(df1, valid_, on='patient_id', how='inner')
df1 = create_training_solution(df1)
print(len(df1))
print(len(sub_df))
score = score(df1, sub_df, 'patient_id')
print(score)

157
157
0.7960391366745849


In [74]:
from itertools import repeat


class SpatialDropout(nn.Module):
    def __init__(self, drop=0.5):
        super(SpatialDropout, self).__init__()
        self.drop = drop
        
    def forward(self, inputs, noise_shape=None):
        """
        @param: inputs, tensor
        @param: noise_shape, tuple
        """
        outputs = inputs.clone()
        if noise_shape is None:
            noise_shape = (inputs.shape[0], *repeat(1, inputs.dim()-2), inputs.shape[-1]) 
        
        self.noise_shape = noise_shape
        if not self.training or self.drop == 0:
            return inputs
        else:
            noises = self._make_noises(inputs)
            if self.drop == 1:
                noises.fill_(0.0)
            else:
                noises.bernoulli_(1 - self.drop).div_(1 - self.drop)
            noises = noises.expand_as(inputs)    
            outputs.mul_(noises)
            return outputs
            
    def _make_noises(self, inputs):
        return inputs.new().resize_(self.noise_shape)


class MLPAttentionNetwork(nn.Module):
 
    def __init__(self, hidden_dim, attention_dim=None):
        super(MLPAttentionNetwork, self).__init__()
 
        self.hidden_dim = hidden_dim
        self.attention_dim = attention_dim
        if self.attention_dim is None:
            self.attention_dim = self.hidden_dim
        # W * x + b
        self.proj_w = nn.Linear(self.hidden_dim, self.attention_dim, bias=True)
        # v.T
        self.proj_v = nn.Linear(self.attention_dim, 1, bias=False)
 
    def forward(self, x):
        """
        :param x: seq_len, batch_size, hidden_dim
        :return: batch_size * seq_len, batch_size * hidden_dim
        """
        # print(f"x shape:{x.shape}")
        batch_size, seq_len, _ = x.size()
        # flat_inputs = x.reshape(-1, self.hidden_dim) # (batch_size*seq_len, hidden_dim)
        # print(f"flat_inputs shape:{flat_inputs.shape}")
        
        H = torch.tanh(self.proj_w(x)) # (batch_size, seq_len, hidden_dim)
        # print(f"H shape:{H.shape}")
        
        att_scores = torch.softmax(self.proj_v(H),axis=1) # (batch_size, seq_len)
        # print(f"att_scores shape:{att_scores.shape}")
        
        attn_x = (x * att_scores).sum(1) # (batch_size, hidden_dim)
        # print(f"attn_x shape:{attn_x.shape}")
        return attn_x

class TimmModel2(nn.Module):
    def __init__(self, backbone, pretrained=False):
        super().__init__()
        self.encoder = timm.create_model(
            backbone,
            in_chans=CFG.in_chans,
            num_classes=CFG.out_dim,
            features_only=False,
            drop_rate=CFG.drop_rate,
            drop_path_rate=CFG.drop_path_rate,
            pretrained=pretrained
        )
        
        hdim = self.encoder.conv_head.out_channels
        self.encoder.classifier = nn.Identity()
        
        self.spatialdropout = SpatialDropout(CFG.dropout)
        self.gru = nn.GRU(hdim, 256, 2, batch_first=True, bidirectional=True)
        self.mlp_attention_layer = MLPAttentionNetwork(512)
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(0),
            nn.LeakyReLU(0.1),
            nn.Linear(256, 1),
        )

    def forward(self, x):  # (bs, nslice, ch, sz, sz)
        bs = x.shape[0]
        x = x.view(bs * CFG.n_slice_per_c, CFG.in_chans, CFG.image_size_cls, CFG.image_size_cls)
        feat = self.encoder(x)
        feat = self.spatialdropout(feat)
        feat = feat.view(bs, CFG.n_slice_per_c, -1)
        feat, _ = self.gru(feat)
        feat = self.mlp_attention_layer(feat) # [bs, 512]
        # feat = feat.contiguous().view(bs * CFG.n_slice_per_c, -1)
        feat = self.head(feat)
        # feat = feat.view(bs, CFG.n_slice_per_c).contiguous()
        # feat = self.mean_layer(feat)
        feat = feat.view(bs, -1)
        return feat


# load cls model
model_cls_path = 'weights'
cls_models = []

for idx_fold in range(1):
    cls_model = TimmModel2(CFG.backbone_cls, pretrained=False)
    load_cls_model_file = os.path.join(model_cls_path, f'filter_stage2_tf_efficientnetv2_s_in21ft1k_fold{idx_fold}_best_078.pth')
    sd = torch.load(load_cls_model_file, map_location='cpu')
    if 'model_state_dict' in sd.keys():
        sd = sd['model_state_dict']
    sd = {k[7:] if k.startswith('module.') else k: sd[k] for k in sd.keys()}
    cls_model.load_state_dict(sd, strict=True)
    cls_model = cls_model.to(device)
    cls_model.eval()
    cls_models.append(cls_model)
print(f'load cls_model succesfull for {len(cls_models)} models!')

/data1/ocr_ie/anaconda3/envs/rsna/lib/python3.8/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21ft1k to current tf_efficientnetv2_s.in21k_ft_in1k.
  model = create_fn(


load cls_model succesfull for 1 models!


In [24]:
# prediction
map_cls = {0: 'liver', 1: 'spleen', 2: 'kidney', 3: 'kidney', 4: 'bowel'}
final_label = {0: 'healthy', 1: 'injury'}
ww_df = pd.read_csv('data/stage1/train_cls.csv')
# val
# patient_id = 10937
# study_id = 53000
patient_id = 27289
study_id = 24214
cid = 4
label = map_cls[cid]
print(label)
s1 = ww_df['patient_id'] == patient_id
s2 = ww_df['study_id'] == study_id
s3 = ww_df['cid'] == cid
s = s1 & s2 & s3
row = ww_df[s]
print(row)
label = row[label].values.tolist()[0]


transforms_valid = albumentations.Compose([
    albumentations.Resize(CFG.image_size_cls, CFG.image_size_cls),
])

images = []
for nid in range(15):
    image = np.load(f"data/stage1/crop/{patient_id}_{study_id}_{cid}_{nid}.npy")
    image = transforms_valid(image=image)['image']
    image = image.transpose(2, 0, 1).astype(np.float32) / 255.
    images.append(image)

images = np.stack(images, 0)
images = torch.tensor(images).float()
images = images.unsqueeze(0)
images = images.to(CFG.device)
print(images.shape)
logits = cls_model(images)
print('pred: ====>', logits)
logits = logits.sigmoid().detach().cpu().numpy()
print('logits: ====>', logits)
print(logits.mean())
print('final label: ====>', final_label[label])

bowel
      patient_id  study_id  bowel  kidney  liver  spleen  cid  fold
6985       27289     24214      0       0      0       0    4     3
torch.Size([1, 15, 6, 224, 224])
pred: ====> tensor([[-5.1040]], device='cuda:0', grad_fn=<ViewBackward0>)
logits: ====> [[0.00603601]]
0.0060360073
final label: ====> healthy


In [1]:
import pandas as pd
df = pd.read_csv('data/stage1/train_cls.csv')
df.head()

,patient_id,study_id,bowel,kidney,liver,spleen,cid,fold
0,10004,51033,0,1,0,1,0,4
1,10004,51033,0,1,0,1,1,4
2,10004,51033,0,1,0,1,2,4
3,10004,51033,0,1,0,1,3,4
4,10004,51033,0,1,0,1,4,4


In [8]:
sub_df = df[df['fold'] == 0]
train_df = pd.read_csv('data/train.csv')
sub_df = pd.merge(train_df, sub_df, on='patient_id', how='inner')
sub_df = sub_df.groupby('patient_id').mean().reset_index()
sub_df = sub_df.drop(['bowel', 'kidney', 'liver', 'spleen'], axis=1)
sub_df.to_csv('data/stage1/fold0_offical.csv', index=False)